In [1]:
import dspy, os
os.environ["OPENAI_API_KEY"] = "EMPTY"  # DSPy expects a key; vLLM ignores it
lm = dspy.LM("openai/meta-llama/Llama-3.1-8B-Instruct",
             api_base="http://127.0.0.1:8000/v1")
dspy.configure(lm=lm)


In [9]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Disable adapters so JSON mode isn't triggered
generate_answer = dspy.Predict(BasicQA, enforce_schema=False)

result = generate_answer(question="What is a fungus?")
print(result.answer)

A fungus is a type of organism.


In [28]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts and psychological insights")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class GenerateSearchQuery(dspy.Signature):
    """Generate a logical rule query based on the context to answer the question."""

    context = dspy.InputField(desc="may contain relevant facts and psychological insights")
    question = dspy.InputField()
    query = dspy.OutputField()


def deduplicate(seq: list[str]) -> list[str]:
    """
        From Raymond Hettinger
        https://twitter.com/raymondh/status/944125570534621185
        Since Python 3.6 Dict are ordered
        Benchmark: https://gist.github.com/peterbe/67b9e40af60a1d5bcb1cfb4b2937b088
    """
    return list(dict.fromkeys(seq))

class MultiHopSearchWithPoT(dspy.Module):
    def __init__(self, num_hops):
        self.num_hops = num_hops
        self.generate_query = dspy.ChainOfThought(GenerateSearchQuery)
        self.generate_answer = dspy.Predict(GenerateAnswer)

    def forward(self, question):
        context = []
        for _ in range(self.num_hops):
            query = self.generate_query(context=context, question=question).query
            context = deduplicate(context + [query])
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [29]:
multi_hop_pot = MultiHopSearchWithPoT(num_hops=2)
question = (
    "Noor is working as a barista at a busy coffee shop. Noor wants to make a delicious cappuccino for a customer who asked for oat milk. Noor grabs a milk pitcher and fills it with oat milk. Noor believes that the milk pitcher contains oatmilk. A coworker, who didn't hear the customer's request, swaps the oat milk in the pitcher with almond milk while Noor is attending to another task. What will Noor do?",
)
multi_hop_pot(question=question).answer

'Noor will serve almond milk.'

In [30]:
dspy.inspect_history(n=10)





[2025-08-14T16:24:26.244540]

System message:

Your input fields are:
1. `context` (str): may contain relevant facts and psychological insights
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `query` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## query ## ]]
{query}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generate a logical rule query based on the context to answer the question.


User message:

[[ ## context ## ]]
N/A

[[ ## question ## ]]
["Noor is working as a barista at a busy coffee shop. Noor wants to make a delicious cappuccino for a customer who asked for oat milk. Noor grabs a milk pitcher and fills it with oat milk. Noor believes that the milk pitcher contains oatmilk. A coworker, who didn't hear the customer's request, swaps the oat milk in the pi

In [39]:
from typing import List, Literal
from pydantic import BaseModel


class Event(BaseModel):
    order: int
    actor: str
    action: str
    belief: str
    location: str | None = None

class ToMSignature(dspy.Signature):
    """Extract implicit reasoning needed for a ToM question, then answer."""
    story: str = dspy.InputField(desc="Short story/problem statement")

    world_facts: List[str] = dspy.OutputField()
    timeline: List[Event] = dspy.OutputField()
    bridging_rules: List[str] = dspy.OutputField()
    answer: str = dspy.OutputField()


class ToMExtractor(dspy.Module):
    def __init__(self):
        super().__init__()
        base = dspy.Predict(ToMSignature)

        def reward_fn(args, pred) -> float:
            score = 0.0
            # 1) final answer present
            if getattr(pred, "answer", "").strip():
                score += 1
    
            return score

        # Try up to N times; stop when score >= threshold
        self.extract = dspy.Refine(
            module=base,
            N=4,
            reward_fn=reward_fn,
            threshold=2.5,   # require any 2–3 of the checks to pass
        )

    def forward(self, story: str):
        return self.extract(story=story)



# 4) Run it on your ToM example
story = (
  "Noor is working as a barista at a busy coffee shop. Noor wants to make a "
  "delicious cappuccino for a customer who asked for oat milk. Noor grabs a "
  "milk pitcher and fills it with oat milk. "
  "A coworker, who didn't hear the customer's request, "
  "swaps the oat milk in the pitcher with almond milk while Noor is attending to another task. "
  "What will Noor do?"
)

extractor = ToMExtractor()
result = extractor(story=story)

# 5) Inspect the structured “reasoning steps”
print("World facts:", *result.world_facts, sep="\n- ")
print("\nTimeline:", *[e.model_dump() for e in result.timeline], sep="\n- ")
print("\nBridging rules:", *result.bridging_rules, sep="\n- ")
print("\nFinal answer:", result.answer)


World facts:
- Noor is a barista at a coffee shop.
- The customer asked for oat milk in their cappuccino.
- The coworker didn't hear the customer's request.
- Noor is attending to another task while the coworker swaps the milk.

Timeline:
- {'order': 1, 'actor': 'Noor', 'action': 'fill the milk pitcher with oat milk', 'belief': 'the customer wants oat milk', 'location': 'the coffee shop'}
- {'order': 2, 'actor': 'the coworker', 'action': 'swap the oat milk with almond milk', 'belief': 'no specific belief', 'location': 'the coffee shop'}
- {'order': 3, 'actor': 'Noor', 'action': 'make a cappuccino with the milk in the pitcher', 'belief': 'the milk in the pitcher is oat milk', 'location': 'the coffee shop'}

Bridging rules:
- If a customer asks for a specific type of milk, the barista should use that type of milk.
- If a coworker swaps the milk without the barista's knowledge, the barista may not know what type of milk is in the pitcher.

Final answer: Noor will likely notice that the mi